In [62]:
import pandas as pd

# Read the table from a text file
df = pd.read_csv("data/A2A.MI.txt", sep="\t", header=0)
df.columns

Index(['date', 'volume', 'high', 'low', 'open', 'close', 'adj_close', 'ticker',
       'ropen', 'rhigh',
       ...
       'rrg_log_returns', 'rrg_cumul', 'rrg_PL_cum_fx', 'rrg_trading_edge',
       'rrg_geometric_expectancy', 'rrg_kelly', 'rrg_constant', 'rrg_concave',
       'rrg_convex', 'rrg_equal_weight'],
      dtype='object', length=122)

In [63]:


# # Select first 15 columns and the 19th column (index 18 because Python is 0-based)
# cols_to_select = list(df.columns[:12]) + [df.columns[14]]

# # Create new DataFrame and reset index
# df = df[cols_to_select].reset_index(drop=True)




# signal_name = 'rrg'
# cols_to_select = list(df.columns[:12]) + [signal_name]
# df = df[cols_to_select].reset_index(drop=True)
df['fx'] = 1
df.columns

Index(['date', 'volume', 'high', 'low', 'open', 'close', 'adj_close', 'ticker',
       'ropen', 'rhigh',
       ...
       'rrg_cumul', 'rrg_PL_cum_fx', 'rrg_trading_edge',
       'rrg_geometric_expectancy', 'rrg_kelly', 'rrg_constant', 'rrg_concave',
       'rrg_convex', 'rrg_equal_weight', 'fx'],
      dtype='object', length=123)

In [64]:
from algoshort.position_sizing import PositionSizing
from algoshort.stop_loss import StopLossCalculator
from algoshort.returns import ReturnsCalculator

In [65]:
# calc = StopLossCalculator(df)
# Method A: Use the ATR-based Stop Loss (Window of 10, Multiplier of 2.0)
# df = calc.atr_stop_loss(price_col = 'close', signal='rbo_100', window=10, multiplier=2.0)

In [69]:
# ret = ReturnsCalculator(df)
# ret.get_returns(df, signal='rbo_100', relative=True, inplace=True)

# pos = PositionSizing(df)
# pos.compare_position_sizing(df=df, signal='rbo_100', price_col='rclose', stop_loss_col='rbo_100_stop_loss', daily_change_col = 'rbo_100_chg1D_fx', inplace=True)

# equity = df[['constant', 'concave', 'convex', 'equal_weight']].tail(1)

def get_equity(df, signal, price_col, stop_loss_col, daily_change_col):
    ret = ReturnsCalculator(df)
    ret.get_returns(df, signal=signal, relative=True, inplace=True)

    pos = PositionSizing(df)
    pos.compare_position_sizing(df=df, signal=signal, price_col=price_col, stop_loss_col=stop_loss_col, daily_change_col = daily_change_col, inplace=True)

    equity = df[['constant', 'concave', 'convex', 'equal_weight']].tail(1)
    return equity


In [75]:
import pandas as pd
import itertools

# 1. Define parameter ranges
windows = [14]
multipliers = [1.5]
signal_names = ['rbo_100', 'rrg', 'rema_50100150', 'rsma_50100150', 'rtt_5020']

# List to store the results from each iteration
all_results = []

# Initialize the calculator once to leverage internal caching
calc = StopLossCalculator(df)

In [71]:
# stop_loss_name = signal_name+'_stop_loss'
# change_name = signal_name+'_chg1D_fx'

In [76]:
for w, m, s in itertools.product(windows, multipliers, signal_names):
    stop_loss_name = s + '_stop_loss'
    change_name = s + '_chg1D_fx'
    # Calculate stop loss
    temp_df = calc.atr_stop_loss(price_col = 'close', signal=s, window=w, multiplier=m)
    
    # Run the evaluation function
    # Expected: equity_calc returns a df with 1 row and 4 columns (e.g., Equity, DD, Sharpe, WinRate)
    metrics_df = get_equity(df=temp_df, signal=s, price_col='close', stop_loss_col=stop_loss_name, daily_change_col=change_name)
    
    # Extract the row as a dictionary and add our parameters
    row_data = metrics_df.iloc[0].to_dict()
    row_data['window'] = w
    row_data['multiplier'] = m
    row_data['signal_names'] = s
    
    all_results.append(row_data)

# 2. Convert all results to a single DataFrame for analysis
results_df = pd.DataFrame(all_results)

In [73]:
# for w, m in itertools.product(windows, multipliers):
#     # Calculate stop loss
#     temp_df = calc.atr_stop_loss(price_col = 'close', signal=signal_name, window=w, multiplier=m)
    
#     # Run the evaluation function
#     # Expected: equity_calc returns a df with 1 row and 4 columns (e.g., Equity, DD, Sharpe, WinRate)
#     metrics_df = get_equity(df=temp_df, signal=signal_name, price_col='close', stop_loss_col=stop_loss_name, daily_change_col=change_name)
    
#     # Extract the row as a dictionary and add our parameters
#     row_data = metrics_df.iloc[0].to_dict()
#     row_data['window'] = w
#     row_data['multiplier'] = m
    
#     all_results.append(row_data)

# # 2. Convert all results to a single DataFrame for analysis
# results_df = pd.DataFrame(all_results)

In [77]:
results_df.sort_values('constant')

,constant,concave,convex,equal_weight,window,multiplier,signal_names
2,936748.0,905122.0,905122.0,1016506.0,14,1.5,rema_50100150
4,947291.0,920664.0,920664.0,1017243.0,14,1.5,rtt_5020
3,960844.0,941266.0,941266.0,1010218.0,14,1.5,rsma_50100150
1,1064730.0,1096458.0,1096458.0,982233.0,14,1.5,rrg
0,1162213.0,1243250.0,1243250.0,945790.0,14,1.5,rbo_100


In [ ]:
for w, m, s in itertools.product(windows, multipliers, signal_names):
    stop_loss_name = s + '_stop_loss'
    change_name = s + '_chg1D_fx'
    # Calculate stop loss
    temp_df = calc.atr_stop_loss(price_col = 'close', signal=s, window=w, multiplier=m)
    
    # Run the evaluation function
    # Expected: equity_calc returns a df with 1 row and 4 columns (e.g., Equity, DD, Sharpe, WinRate)
    metrics_df = get_equity(df=temp_df, signal=s, price_col='close', stop_loss_col=stop_loss_name, daily_change_col=change_name)
    
    # Extract the row as a dictionary and add our parameters
    row_data = metrics_df.iloc[0].to_dict()
    row_data['window'] = w
    row_data['multiplier'] = m
    row_data['signal_names'] = s
    
    all_results.append(row_data)

# 2. Convert all results to a single DataFrame for analysis
results_df = pd.DataFrame(all_results)